In [1]:
import tarfile
import urllib
#import urllib2
from os.path import isfile, isdir
from tqdm import tqdm_notebook as tqdm

import os
#import cv2
import h5py
import math
import numpy as np
from PIL import Image
from scipy import misc
import random as rand
from operator import add
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.patches
import numpy as np
import tarfile
import urllib
from IPython.display import display, Image
from scipy import ndimage

import scipy.io as sio
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
import sys
import pandas as pd
%matplotlib inline
from sklearn.model_selection import train_test_split
import glob
import csv

import re
#import nltk
#from nltk.corpus import stopwords
#import nltk.data
from sklearn.utils import shuffle
import zipfile

# Load data

In [2]:
train_q1 = np.load('data/train_q1.npy')
train_q2 = np.load('data/train_q2.npy')
train_y = np.load('data/train_y.npy')
cv_q1 = np.load('data/cv_q1.npy')
cv_q2 = np.load('data/cv_q2.npy')
cv_y = np.load('data/cv_y.npy')
test_q1 = np.load('data/test_q1.npy')
test_q2 = np.load('data/test_q2.npy')

In [3]:
print(len(train_q1), len(train_q2), len(train_y), len(cv_q1), len(cv_q2), len(cv_y), len(test_q1), len(test_q2))

323432 323432 323432 80858 80858 80858 2345796 2345796


# Load w2v

In [4]:
with open("data/w2v", 'rb') as f:
    w2v = pickle.load(f)

# Transform question to vector

In [14]:
def question_to_vectors_w2v(question, length = 20):
    vectors = np.zeros((length, 300), dtype = "float32")
    
    number_of_words = 0
    
    for idx, word in enumerate(question):
        if word in w2v:
            vector = w2v[word]
            vectors[number_of_words] = vector
            number_of_words += 1
        if number_of_words == length:
            break
    return vectors

In [15]:
question_to_vectors_w2v(["i", "love", "you"])

array([[-0.08573538, -0.00742298,  0.03451684, ...,  0.01071692,
        -0.06754909, -0.00229648],
       [ 0.03863575, -0.05712964,  0.00970471, ..., -0.07434177,
        -0.04614317,  0.02435334],
       [ 0.08773035,  0.00566679,  0.03253159, ..., -0.09108845,
        -0.05708769,  0.04659363],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [16]:
#testing
x = question_to_vectors_w2v(["i", "love", "you"])
print(x)
print(type(x), x.shape)

[[-0.08573538 -0.00742298  0.03451684 ...,  0.01071692 -0.06754909
  -0.00229648]
 [ 0.03863575 -0.05712964  0.00970471 ..., -0.07434177 -0.04614317
   0.02435334]
 [ 0.08773035  0.00566679  0.03253159 ..., -0.09108845 -0.05708769
   0.04659363]
 ..., 
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]]
<class 'numpy.ndarray'> (20, 300)


In [ ]:
#testing
q1 = train_q1[1000]
print(q1)
x1 = question_to_vectors_w2v(q1)
print(x1)
print(x1.shape)

# Generate data_batch

In [17]:
def data_generator_w2v(questions1, questions2, y, batch_size=1024):
    
    sample_size = len(y)
    batch_slices = [slice(i, i + batch_size) for i in range(0, sample_size, batch_size)]
    
    while True:
        for batch in batch_slices:
            batch_new_questions1 = []
            batch_new_questions2 = []
            batch_y = y[batch]

            for question1 in questions1[batch]:
                new_question1 = question_to_vectors_w2v(question1)
                batch_new_questions1.append(new_question1)
            
            for question2 in questions2[batch]:
                new_question2 = question_to_vectors_w2v(question2)
                batch_new_questions2.append(new_question2)

            yield [np.array(batch_new_questions1), np.array(batch_new_questions2)], np.array(batch_y)

# Train model

In [18]:
from keras import callbacks
from keras.models import Sequential
from keras.callbacks import EarlyStopping
import tensorflow as tf
import keras
from keras import layers

Using TensorFlow backend.


In [19]:
def tower(output_dim, num_of_layers, activation, dropout, input_dim=(20, 300)):
    x = layers.Input(input_dim)
    
    if num_of_layers == 1:

        rep = layers.recurrent.LSTM(output_dim,  
                                    input_shape=input_dim,  
                                    return_sequences=False, 
                                    activation=activation,
                                    dropout=dropout)(x)

        rep = layers.Dropout(dropout)(rep)
    
    if num_of_layers == 2:
    
        rep = layers.recurrent.LSTM(output_dim,  
                                    input_shape=input_dim,  
                                    return_sequences=True, 
                                    activation=activation,
                                    dropout=dropout)(x)

        rep = layers.Dropout(dropout)(rep)
        
        rep = layers.recurrent.LSTM(output_dim)(rep)

        
    return x, rep

In [20]:
def two_towers_operation(left, right, method):
    if method == "concat":
        return layers.concatenate([left, right])
    elif method == "multiply":
        return layers.multiply([left, right])
    elif method == "ave":
        return layers.average([left, right])
    elif method == "dif":
        return layers.merge([left, right], mode=lambda x: x[0] - x[1], output_shape=lambda x: x[0])

In [23]:
def train_model():
    
    in1, left = tower(100, 1, "softsign", 0.5)
    in2, right = tower(100, 1, "softsign", 0.5)

    #rep = merge_two_towers(left, right, "concat")
    #rep = two_towers_operation(left, right, "multiple")
    #rep = two_towers_operation(left, right, "ave")
    rep = two_towers_operation(left, right, "dif")
    
    rep = layers.Dropout(0.5)(rep)
    
    #output = layers.Dense(2, activation='softmax')(rep)
    output = layers.Dense(2, activation='sigmoid')(rep)

    model = keras.models.Model(inputs=[in1, in2], outputs=[output])
    
    model.compile(loss='sparse_categorical_crossentropy', 
                  optimizer='adam',
                  metrics=['accuracy'])
    
    model.fit_generator(data_generator_w2v(train_q1, train_q2, train_y),
                    steps_per_epoch=len(train_y)// 1024, epochs=10, validation_data=data_generator_w2v(cv_q1, cv_q2, cv_y),
                    validation_steps = len(cv_y)// 1024)
    return model

In [24]:
model = train_model()

/usr/lib/python3.6/site-packages/ipykernel/__main__.py:9: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/jupyter/.local/lib/python3.6/site-packages/keras/legacy/layers.py:460: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[1024,20,300]
	 [[Node: lstm_3/Tile_2 = Tile[T=DT_FLOAT, Tmultiples=DT_INT32, _device="/job:localhost/replica:0/task:0/gpu:0"](lstm_3/ExpandDims_2, lstm_3/stack_2)]]
	 [[Node: mul_3/_71 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_2408_mul_3", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'lstm_3/Tile_2', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/lib/p